In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt

from skimage.transform import resize

import keras

Using TensorFlow backend.


In [11]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    

    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    
    print("Study Description: {0}".format(ds.StudyDescription))

    print("Modality: {0}".format(ds.Modality))
    if ds.Modality != "DX":
        print(">> Warning! Modality must be DX. Can't make a prediction.")
    print("Patient Sex: {0}".format(ds.PatientSex))
    if ds.PatientSex not in ['M', 'F']:
        print(">> Warning! Unsupported Patient_Sex value in DICOM file.")
    print("Patient Age: {0}".format(ds.PatientAge))
    min_age = 11
    max_age = 76
    if int(ds.PatientAge) < 20 or int(ds.PatientAge) > 80:
        print(">> Warning! Supported range for patient age is between {0} and {1} years old.".format(min_age, max_age))
    print("Body Part Examined: {0}".format(ds.BodyPartExamined))
    if ds.BodyPartExamined != 'CHEST':
        print(">> Warning! Unsupported value for Body_Part_Examined field. Can't make a prediction.")
    print("Patient Position: {0}".format(ds.PatientPosition))
    if ds.PatientPosition not in ['AP', 'PA']:
        print(">> Warning! Unsupported value for Patient_Position field. Can't make a prediction.")
    
    
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    
    #rescale 1./255
    proc_img = img / 255.0  
    
    #omar:normalize
    proc_img = proc_img - img_mean
    proc_img = proc_img / img_std
    
    #oamr: scale
    proc_img = resize(proc_img, (img_size))
    
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    model = keras.models.model_from_json(loaded_model_json)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    prob_pred_Y = model.predict(img, batch_size = 1, verbose = True)
    prediction = (prob_pred_Y >= thresh)
    return prediction, prob_pred_Y 

In [16]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"#path to saved model
weight_path = "xray_class_my_model.best.hdf5"#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)#loads model
thresh = 0.6#loads the threshold they chose for model classification 

In [17]:


# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    
    #plt.imshow(img, cmap = 'bone')
    
    prediction, prob_pred_Y = predict_image(my_model,img_proc,thresh)
    print("{0}. Prob: {1}".format(prediction, prob_pred_Y))
    print("========================================================")

Load file test1.dcm ...
Study Description: No Finding
Modality: DX
Patient Sex: M
Patient Age: 81
>> Warning! Supported range for patient age is between 11 and 76 years old.
Body Part Examined: CHEST
Patient Position: PA


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


1/1 [==============================] - 1s 1s/step
[[False]]. Prob: [[0.58762735]]
Load file test2.dcm ...
Study Description: Cardiomegaly
Modality: DX
Patient Sex: M
Patient Age: 58
Body Part Examined: CHEST
Patient Position: AP
1/1 [==============================] - 1s 1s/step
[[False]]. Prob: [[0.5844818]]
Load file test3.dcm ...
Study Description: Effusion
Modality: DX
Patient Sex: M
Patient Age: 77
Body Part Examined: CHEST
Patient Position: AP
1/1 [==============================] - 1s 1s/step
[[False]]. Prob: [[0.63891023]]
Load file test4.dcm ...
Study Description: No Finding
Modality: DX
Patient Sex: M
Patient Age: 81
>> Warning! Supported range for patient age is between 11 and 76 years old.
Body Part Examined: RIBCAGE
>> Warning! Unsupported value for Body_Part_Examined field. Can't make a prediction.
Patient Position: PA
1/1 [==============================] - 1s 1s/step
[[False]]. Prob: [[0.58762735]]
Load file test5.dcm ...
Study Description: No Finding
Modality: CT
>> Warni